In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
lv = LV.from_pyvista_read("../sample_files/lvhexmesh.vtk") # load reference mesh (can also be done from other methods)
lv.tetrahedralize(backend=TETRA_BACKEND.TETGEN, order=1, mindihedral=0.0, minratio=2.0)
lv.mesh

UnstructuredGrid,Information
N Cells,365443
N Points,95495
X Bounds,"-2.245e+01, 5.452e+01"
Y Bounds,"-4.001e+01, 4.336e+01"
Z Bounds,"-4.906e+01, 4.008e+01"
N Arrays,0


In [3]:
lv.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2=0.5,
                          a3=0.3,
                          a4=75,
                          a5=130,
                          
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)
lv.plot("surface", scalars=LV_MESH_DATA.SURFS_DETAILED.value,
        cmap="tab20_r")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(position=(49999.999999…

In [4]:
lv.compute_fibers(surfRegionsIds=LV_FIBER_MODES.LDRB_1,
                    fiber_space="P_1",
                    alpha_endo_lv=60,  # Fiber angle on the endocardium
                    alpha_epi_lv=-60,  # Fiber angle on the epicardium
                    beta_endo_lv=0,  # Sheet angle on the endocardium
                    beta_epi_lv=0,  # Sheet angle on the epicardium
                    markers={},
                    ldrb_kwargs={},
                    save_xdmfs=False,
                    xdmfs_dir=None,
                    xdmfs_basename=None,
                    del_generated_files=True,
                    )

Calculating scalar fields
Compute scalar laplacian solutions with the markers: 
epi: 2
lv: 1
base: 22
  Num vertices: 95495
  Num cells: 365443
Solving linear variational problem.
  Apex coord: (45.03, -27.18, -39.29)
Solving linear variational problem.
Solving Laplace equation
 epi = 1, lv = 0
Solving linear variational problem.
 lv = 1, epi = 0
Solving linear variational problem.

Calculating gradients
Compute fiber-sheet system
Angles: 
alpha: 
 endo_lv: 60
 epi_lv: -60
 endo_septum: 60
 epi_septum: -60
 endo_rv: 60
 epi_rv: -60
beta: 
 endo_lv: 0
 epi_lv: 0
 endo_septum: 0
 epi_septum: 0
 endo_rv: 0
 epi_rv: 0


In [5]:
lv.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.85)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [17]:
lv.mesh.save("../sample_files/lvhexmesh_tetcoarse_counterpart.vtk")

In [6]:
lv_hex = LV.from_pyvista_read("../sample_files/lvhexmesh.vtk")

In [7]:
lv_src = lv 
lv_dst = lv_hex

In [8]:
# from this data
X = lv_src.points()
Y1 = lv_src.get(GEO_DATA.MESH_POINT_DATA, LV_FIBERS.F0)
Y2 = lv_src.get(GEO_DATA.MESH_POINT_DATA, LV_FIBERS.S0)
Y3 = lv_src.get(GEO_DATA.MESH_POINT_DATA, LV_FIBERS.N0)
Y = np.hstack((Y1, Y2, Y3))
# to this data
XI = lv_dst.points()

In [9]:
from sklearn.neural_network import MLPRegressor
MLPR = MLPRegressor((100,),
                    early_stopping=True,
                    validation_fraction=0.25,
                    )
MLPR = MLPR.fit(X, Y)
YI = MLPR.predict(XI)

In [13]:
lv_dst.mesh.point_data[LV_FIBERS.F0.value] = YI[:, :3]
lv_dst.mesh.point_data[LV_FIBERS.S0.value] = YI[:, 3:6]
lv_dst.mesh.point_data[LV_FIBERS.N0.value] = YI[:, 6:]

In [14]:
lv_dst.identify_base_and_apex_surfaces() # computes normal
lv_dst.compute_fiber_angles()

In [15]:
lv_dst.mesh.save("./testhexfibers.vtk")

In [16]:
lv_dst.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.8)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [18]:
lv_dst.mesh.save("./lvhexmesh_with_fibers.vtk")